In [ ]:
# Librairies & paramètres
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import plotly.express as px
import plotly.graph_objects as go

chemin_actuel = os.getcwd()
dossier_parent = os.path.abspath(os.path.join(chemin_actuel, os.pardir))
dossier_parent = os.path.abspath(os.path.join(dossier_parent, os.pardir))

param = {

    # Direction vers lmes fichiers SRA
    "GeoSRA DIR IN" :dossier_parent+"/data/raw/geo-sra-ecological.parquet.gzip",
    "GeoSRA DIR OUT":dossier_parent+"/data/processed/geosra_coordprec.parquet.gzip",
    "Country DIR"   :'naturalearth_lowres',
    
    # Seuil minimal de précision des coordonées
    "Seuil LonLatAc":2,

    # Taille des figures
    "Width Figure"  :800,
    "Height Figure" :600,

    # Couleurs
    "Georef Color"  :{
                    "(?)"        :'lightgrey',
                    "Has LonLat" : 'lightgreen',
                    "Has Country": 'yellow',
                    "NR"         : 'lightcoral'
    },

    "Binary Color"  :{
                    "(?)"  :'lightgrey',
                    "True" :'lightcoral',
                    "False":"lightgreen"
    }

}

In [ ]:
# Ouverture des fichiers
geosra_pd = pd.read_parquet(param["GeoSRA DIR IN"])
file = gpd.datasets.get_path(param["Country DIR"])
countries = gpd.read_file(file)
geosra_pd.head()

In [ ]:
# Création de la colonne release_year et Histogramme par années
geosra_pd["release_year"] = geosra_pd["releasedate"].dt.year

fig = px.histogram(geosra_pd, x="release_year") 
fig.update_layout(
    width=param["Width Figure"],
    height=param["Height Figure"]
)            
fig.show()

In [ ]:
# Répartition des types de géoréférencement par années

df = geosra_pd[["release_year","has_latlon","geo_loc_name_country_calc","rg_country"]]
df["georef"] = 'NR'
df.loc[df["has_latlon"] == True, "georef"] = 'Has LonLat'
df.loc[((df["geo_loc_name_country_calc"].notnull()) | (df["rg_country"] != '') & df["has_latlon"] == False), "georef"] = 'Has Country'

fig = px.histogram(df, x="release_year", color="georef",
                color_discrete_map=param["Georef Color"]) 
fig.update_layout(
    width=param["Width Figure"],
    height=param["Height Figure"]
)            
fig.show()    

In [ ]:
# Répartition de la précision des coorodnées
def nombre_decimales(nombre):
    if nombre == np.NaN:
        return np.NaN
    chaine = str(nombre)
    index_point = chaine.find('.')
    if index_point == -1:
        return np.NaN
    return len(chaine) - index_point - 1
geosra_pd["latitude_precision"] = geosra_pd["latitude"].apply(nombre_decimales)
geosra_pd["longitude_precision"] = geosra_pd["longitude"].apply(nombre_decimales)

df = geosra_pd[["longitude_precision","latitude_precision"]]
fig = go.Figure()
fig.add_trace(go.Box(y=np.array(df["longitude_precision"]), boxmean='sd', boxpoints=False, name="Longitude Precision"))
fig.add_trace(go.Box(y=np.array(df["latitude_precision"]), boxmean='sd', boxpoints=False, name="Latitude Precision"))
fig.update_layout(
    width=param["Width Figure"],
    height=param["Height Figure"]
)   
fig.show()

In [ ]:
# Calcul des imprécisions par années
thr = param["Seuil LonLatAc"]
df = geosra_pd[["release_year","latitude_precision","longitude_precision"]]
df["imprécis"] = 'False'
df.loc[(df["latitude_precision"] <= thr) | (df["longitude_precision"] <= thr), "imprécis"] = 'True'
fig = px.histogram(df, x="release_year", color="imprécis",
                color_discrete_map=param["Binary Color"]) 
fig.update_layout(
    width=param["Width Figure"],
    height=param["Height Figure"]
)            
fig.show()

In [ ]:
# Enregistrement des données potentiellement exploitables
geosra_pd.to_parquet(param["GeoSRA DIR OUT"])